In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.options.display.max_rows = 10
pd.options.display.max_columns = 500

In [2]:
DIVERSITY_UTILITY = 0.25
FRAC_ADMIT = 0.5


###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['key'] = df['R'].astype(str) + "_"  + df['T'].astype(str)

df_stratum_utility = df[['R','T','B_p']].groupby(['R','T']).mean().reset_index()
df_stratum_utility['stratum_utility'] = (df_stratum_utility['B_p'] + DIVERSITY_UTILITY * df_stratum_utility['R']).round(2)
df_stratum_utility['key'] = df_stratum_utility['R'].astype(str) + "_" + df_stratum_utility['T'].astype(str)

df = df.merge(df_stratum_utility[['stratum_utility','key']], on='key')
df['ml_outcomes'] = df['stratum_utility']

In [3]:
#df['B_p_reject'] = np.random.randint(0,2,size=(len(df)))

In [4]:
df['B_p_reject']

0        0
1        0
2        0
3        0
4        1
        ..
99995    1
99996    1
99997    1
99998    0
99999    1
Name: B_p_reject, Length: 100000, dtype: int64

In [5]:
df['B_p_stratum'] = df['B_p'].astype(str) + df['B_p_reject'].astype(str)



In [6]:
df['B_p_stratum'].value_counts()

00    55959
11    35442
10     8599
Name: B_p_stratum, dtype: int64

In [7]:
# = 
#df['B_p_reject'] = df['B_p']

In [8]:
FRAC_ADMIT = 0.5#df[['A']].sum()/len(df)


In [9]:
FRAC_ADMIT

0.5

In [10]:
len(df)

100000

In [11]:
#df['ml_outcomes'] = df['R']

In [12]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_black','T_white','B_p']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_black,T_white,B_p
98627,0,20,0.00,15,20,0
99515,0,19,0.00,17,19,0
99514,0,19,0.00,18,19,0
99920,0,15,0.00,13,15,0
99513,0,19,0.00,18,19,0
...,...,...,...,...,...,...
99369,1,92,1.25,92,105,1
99370,1,92,1.25,92,105,1
99371,1,92,1.25,92,108,1
99943,1,85,1.25,85,98,1


In [13]:
#df['ml_outcomes'] = df['ml_outcomes'] + 1*df['R']

In [14]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','B_p','B_p_reject','ml_outcomes']].groupby(['R','T','B_p','B_p_reject']).sum().reset_index()

In [15]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes','B_p','B_p_reject']].groupby(['R','T','B_p','B_p_reject']).count().reset_index()
df_count.columns = ['R','T','B_p','B_p_reject','Count']
df_count['N'] = df_count['Count']

In [16]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [17]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,B_p,B_p_reject,ml_outcomes,N
0,469,0,0,14,0,0,0.00,1
1,464,1,0,15,0,0,0.00,2
2,468,2,0,16,0,0,0.00,2
7,465,7,0,20,0,0,0.00,11
5,467,5,0,18,0,0,0.00,6
...,...,...,...,...,...,...,...,...
100,4,100,0,52,0,0,559.80,1244
144,3,144,0,66,1,1,568.71,801
147,2,147,0,67,1,1,570.13,781
129,1,129,0,61,1,1,577.28,902


### Setup optimization problem 

In [18]:
from ortools.linear_solver import pywraplp


In [19]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [20]:
len(dff[['R','T','B_p','B_p_reject']])

470

In [21]:
dff

,level_0,index,R,T,B_p,B_p_reject,ml_outcomes,N
0,469,0,0,14,0,0,0.00,1
1,464,1,0,15,0,0,0.00,2
2,468,2,0,16,0,0,0.00,2
3,421,3,0,17,0,0,1.20,3
4,441,4,0,17,1,1,0.80,2
...,...,...,...,...,...,...,...,...
465,392,465,1,95,1,1,2.50,2
466,393,466,1,97,1,1,2.50,2
467,416,467,1,98,1,0,1.25,1
468,417,468,1,99,1,1,1.25,1


In [22]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'],row['B_p'], row['B_p_reject'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [23]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [24]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total applicants cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

50000


In [25]:
# Now we have one constraint
solver.NumConstraints()

1

## Add Equalized Odds Constraints

In [26]:
## Make sure that you have to add all people in B_p stratum or none
## i.e. you can't add only people who pass boards and reject those who fail boards from same T, R stratum
didntexist, exists = 0, 0 

for ix, row in dff.iterrows():
    
    
    var1 = vars_cache[(row['R'],row['T'],row['B_p'], row['B_p_reject'])]
    key2s = [(row['R'],row['T'], 1-row['B_p'], 1-row['B_p_reject']), (row['R'],row['T'], row['B_p'], 1-row['B_p_reject']), (row['R'],row['T'], 1-row['B_p'], row['B_p_reject'])]
    
    for key2 in key2s:
        constrain_bp = solver.Constraint(0.0, 0.0)
        
        if key2 not in vars_cache:
            didntexist+=1
            continue
            
        var2 = vars_cache[key2]
        
        constrain_bp.SetCoefficient(var1, -1.0)
        constrain_bp.SetCoefficient(var2, 1.0)
        exists+=1

didntexist, exists

(588, 822)

In [27]:
white_pass_boards_pass_boards_reject = []
white_fail_boards_pass_boards_reject = []
black_pass_boards_pass_boards_reject = []
black_fail_boards_pass_boards_reject = []
white_pass_boards_fail_boards_reject = []
white_fail_boards_fail_boards_reject = []
black_pass_boards_fail_boards_reject = []
black_fail_boards_fail_boards_reject = []


for key in vars_cache:
    r, t, b_p, b_p_reject = key
    if b_p == 1 and r==0 and b_p_reject==1:
        white_pass_boards_pass_boards_reject.append(key)
    elif b_p == 0 and r==0 and b_p_reject==1:
        white_fail_boards_pass_boards_reject.append(key)
    elif b_p == 1 and r==1 and b_p_reject==1:
        black_pass_boards_pass_boards_reject.append(key)
    elif b_p == 0 and r==1 and b_p_reject==1:
        black_fail_boards_pass_boards_reject.append(key)
        
    if b_p == 1 and r==0 and b_p_reject==0:
        white_pass_boards_fail_boards_reject.append(key)
    elif b_p == 0 and r==0 and b_p_reject==0:
        white_fail_boards_fail_boards_reject.append(key)
    elif b_p == 1 and r==1 and b_p_reject==0:
        black_pass_boards_fail_boards_reject.append(key)
    elif b_p == 0 and r==1 and b_p_reject==0:
        black_fail_boards_fail_boards_reject.append(key)

len(white_pass_boards_pass_boards_reject),len(white_fail_boards_pass_boards_reject),len(black_pass_boards_pass_boards_reject),len(black_fail_boards_pass_boards_reject)




(99, 0, 78, 0)

In [28]:
NUM_TOTALS = {}
df_totals = dff[['N','R','B_p','B_p_reject']].groupby(['R','B_p','B_p_reject']).sum().reset_index()
for ix, row in df_totals.iterrows():
    NUM_TOTALS[(row['R'],row['B_p'],row['B_p_reject'])] = row['N']
    
N_IN_STRATAS = {}
for ix, row in dff.iterrows():
    N_IN_STRATAS[(row['R'],row['T'],row['B_p'],row['B_p_reject'])] = row['N']

In [29]:
# Now we have one constraint
solver.NumConstraints()

1411

In [30]:
#Of those who pass the boards exams, pass boards exams if rejected
#Frac white admitted and frac black admitted should be the same

constrain_pass_boards_pass_boards_reject = solver.Constraint(0.0, 0.0)

for key in white_pass_boards_pass_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_pass_boards_pass_boards_reject.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_pass_boards_pass_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_pass_boards_pass_boards_reject.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


In [31]:
#Of those who fail the boards exams, pass boards exams if rejected
#Frac white admitted and frac black admitted should be the same

constrain_fail_boards_pass_boards_reject = solver.Constraint(0.0, 0.0)

for key in white_fail_boards_pass_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_fail_boards_pass_boards_reject.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_fail_boards_pass_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_fail_boards_pass_boards_reject.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


In [32]:
constrain_pass_boards_fail_boards_reject = solver.Constraint(0.0, 0.0)

for key in white_pass_boards_fail_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_pass_boards_fail_boards_reject.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_pass_boards_fail_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_pass_boards_fail_boards_reject.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))

    
#Of those who fail the boards exams, fail boards exams if rejected
#Frac white admitted and frac black admitted should be the same

constrain_fail_boards_fail_boards_reject = solver.Constraint(0.0, 0.0)

for key in white_fail_boards_fail_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_fail_boards_fail_boards_reject.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_fail_boards_fail_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_fail_boards_fail_boards_reject.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


## Add constraints on people who fail boards exams

## Solve linear program

In [33]:
solver.ABNORMAL

4

In [34]:
status = solver.Solve()


In [35]:
status

0

In [36]:
solver.OPTIMAL

0

In [37]:
applicant_stratum

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [38]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [39]:
df_decisions

,row_id,decision
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
465,465,1.0
466,466,1.0
467,467,0.0
468,468,1.0


In [40]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,B_p,B_p_reject,ml_outcomes,N,row_id,decision
135,0,135,0,63,1,1,585.42,887,135,1.0
129,1,129,0,61,1,1,577.28,902,129,1.0
147,2,147,0,67,1,1,570.13,781,147,1.0
144,3,144,0,66,1,1,568.71,801,144,1.0
100,4,100,0,52,0,0,559.80,1244,100,0.0
...,...,...,...,...,...,...,...,...,...,...
7,465,7,0,20,0,0,0.00,11,7,0.0
6,466,6,0,19,0,0,0.00,10,6,0.0
5,467,5,0,18,0,0,0.00,6,5,0.0
2,468,2,0,16,0,0,0.00,2,2,0.0


In [41]:
xxx_ = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx_.sort_values(by='T')

,level_0,index,R,T,B_p,B_p_reject,ml_outcomes,N,row_id,decision
257,453,257,1,11,0,0,0.25,1,257,0.0
258,448,258,1,12,0,0,0.50,2,258,0.0
0,469,0,0,14,0,0,0.00,1,0,0.0
259,454,259,1,14,0,0,0.25,1,259,0.0
260,434,260,1,15,0,0,1.00,4,260,0.0
...,...,...,...,...,...,...,...,...,...,...
252,401,252,0,114,1,1,2.00,2,252,0.0
253,398,253,0,115,1,1,2.00,2,253,0.0
254,431,254,0,116,1,1,1.00,1,254,0.0
255,432,255,0,120,1,1,1.00,1,255,0.0


In [42]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,B_p,B_p_reject,ml_outcomes,N,row_id,decision
135,0,135,0,63,1,1,585.42,887,135,1.0
129,1,129,0,61,1,1,577.28,902,129,1.0
147,2,147,0,67,1,1,570.13,781,147,1.0
144,3,144,0,66,1,1,568.71,801,144,1.0
100,4,100,0,52,0,0,559.80,1244,100,0.0
...,...,...,...,...,...,...,...,...,...,...
7,465,7,0,20,0,0,0.00,11,7,0.0
6,466,6,0,19,0,0,0.00,10,6,0.0
5,467,5,0,18,0,0,0.00,6,5,0.0
2,468,2,0,16,0,0,0.00,2,2,0.0


In [43]:
xxx.to_csv('./decision.csv')

In [44]:
solution = pd.read_csv('./decision.csv')

In [45]:
len(xxx)

470

In [46]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)+'_'+df['B_p'].astype(str)+'_'+df['B_p_reject'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)+'_'+xxx['B_p'].astype(str)+'_'+xxx['B_p_reject'].astype(str)



In [47]:
admit_decisions = df.merge(xxx[['key','decision']],how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT

FRAC_BLACK_POLICY = (admit_decisions['R'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision_random']).sum()

In [48]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Principal Fairness',str(FRAC_BLACK_POLICY),str(SUM_BP_POLICY)))
file.close()

In [49]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.49999999999999983